<a href="https://colab.research.google.com/github/youkyung2/dataAnalysis/blob/master/%EC%9E%91%EC%97%85%ED%98%95_%EC%A0%9C2%EC%9C%A0%ED%98%95_HWANG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 출력을 원하실 경우 print() 활용
# 예) print(df.head())

# 답안 제출 예시
# 수험번호.csv 생성
# DataFrame.to_csv("0000.csv", index=False)

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

x = pd.read_csv('data/X_train.csv', encoding='UTF-8')
y = pd.read_csv('data/y_train.csv', encoding='UTF-8')
x_test = pd.read_csv('data/X_test.csv', encoding='UTF-8')
cust_id = pd.DataFrame(x_test['cust_id'])

data = pd.merge(x, y, on = 'cust_id')
#print(data.shape)

# print(data.info())
# print(data.isnull().sum())
# print(data['환불금액'].describe())

data['환불금액'] = data['환불금액'].fillna(0)
x_test['환불금액'] = x_test['환불금액'].fillna(0)
# print(data.isnull().sum())

# 이상치 탐색도 하면 좋겠다!
# feature importance 구해보면 좋겠다!
#print(data.head(3))

obj_col = data.select_dtypes(include=np.object).columns
#print(obj_col)

# print(data['주구매상품'].value_counts())
# print(data['주구매지점'].value_counts())

product = pd.get_dummies(data['주구매상품'])
producttest = pd.get_dummies(x_test['주구매상품'])
place = pd.get_dummies(data['주구매지점'])
placetest = pd.get_dummies(x_test['주구매지점'])

x = data.drop(['cust_id','gender'],axis=1)
x_dum = pd.get_dummies(x)
feature_name_lst = x_dum.columns
#print(feature_name_lst)

test_dum = pd.get_dummies(x_test.drop(['cust_id'],axis=1))

missing_cols = set(x_dum.columns) - set(test_dum.columns)
print(missing_cols)

for c in missing_cols :
  test_dum[c] = 0
test_dum = test_dum[x_dum.columns]
# print(test_dum)

data = data.drop(columns=['cust_id','주구매상품','주구매지점'], axis=1)
x_test = x_test.drop(columns=['cust_id','주구매상품','주구매지점'], axis=1)

# print(data.head(3))
# print(data.shape)
data = pd.concat([data, product], axis = 1)
x_test = pd.concat([x_test, producttest], axis = 1)
#print(data.head(3))

data = pd.concat([data, place], axis = 1)
x_test = pd.concat([x_test, placetest], axis = 1)
#print(data.shape)


# 피쳐스케일링
x = data.drop(columns=['gender'], axis=1)
y = data['gender']
# print(x)
# print(y)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_scaled = sc.fit_transform(x_dum)
x_scaledtest = sc.fit_transform(test_dum)
# print(x_scaled)

# 훈련셋, 데이터셋 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=20)

# 모델 생성하고 roc-auc 로 평가하기
from sklearn.metrics import roc_auc_score

# 1. 의사결정나무
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
dt_score = roc_auc_score(dt_pred, y_test)
print('의사결정나무 score : {0:.3f}'.format(dt_score))

# 2. 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_score = roc_auc_score(lr_pred, y_test)
print('로지스틱 score : {0:.3f}'.format(lr_score))

# 3. XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=20)
xgb.fit(X_train, y_train)
#GridSearchCV 를 통해 초매개변수 최적화
from sklearn.model_selection import GridSearchCV
params = {
	'n_estimators' : [200,300],
	'max_depth' : [2,3,5],
	'learning_rate' : [0.1]
}
grid = GridSearchCV(xgb, param_grid = params, scoring='accuracy', cv=3)
grid.fit(X_train, y_train)
print('XGB 최적 파라미터 : ',grid.best_params_)
print('XGB 최고 score : ',grid.best_score_)
best_grid = grid.best_estimator_
xgb_pred = best_grid.predict(X_test)
xgb_score = roc_auc_score(xgb_pred, y_test)
print('XGBoost score : {0:.3f}'.format(xgb_score))

xgb_proba = best_grid.predict_proba(X_test)[:,1]
# print(xgb_proba)
#rint(best_grid.feature_importances_)

# 4. 랜덤포레스트
# from sklearn.ensemble import RandomForestClassifier
# rf_clf = RandomForestClassifier(random_state=20)
# rf_clf.fit(X_train, y_train)
# rf_pred = rf_clf.predict(X_test)
# rf_score = roc_auc_score(rf_pred, y_test)
# print('랜덤포레스트 score : {0:.3f}'.format(rf_score))
# # GridSearchCV 를 통해 초매개변수 최적화
# from sklearn.model_selection import GridSearchCV
# params = {
# 	#'n_estimators' : [100,200,300],
# 	'max_depth' : [2,3,5],
# 	'min_samples_split' : [2,3,5],
# 	'min_samples_leaf' : [1,5,8]
# }
# grid = GridSearchCV(rf_clf, param_grid = params, scoring ="accuracy", cv = 5)
# grid.fit(X_train, y_train)
# print('grid 최적 파라미터 : ',grid.best_params_)
# print('grid 최고 score : ',grid.best_score_)
# best_grid2 = grid.best_estimator_

# rf_pred = best_grid2.predict(X_test)
# rf_score = roc_auc_score(rf_pred, y_test)
# print('매개변수 최적화 후 랜덤포레스트 score : {0:.3f}'.format(rf_score))


# 모형앙상블... 
# 1. 보팅
# 2. 배깅 (랜덤포레스트)
# 3. XGBoost

# 테스트데이터에 적용
# 전처리 끝난 test : x_scaledtest

result_proba = best_grid.predict_proba(x_scaledtest)[:,1]
print(result_proba)
cust_id['gender'] = result_proba

print(cust_id.head(3))

result = cust_id.rename(columns={'cust_id' : 'custid'})

print(result.head(4))

result.to_csv('data/XXX.csv')

tee = pd.read_csv('data/XXX.csv')
print(ttt.head(3))
